<a href="https://colab.research.google.com/github/Vishva-D/-Solve4aBillion-Challenge/blob/main/html_integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# State-wise COVID-19 Tracker | India
#### Visualisation of Choropleth map to show the details of COVID-19 cases for Indian states and union territories. It displays - total cases, active cases, and total deaths in each Indian state. We tried to render a map that is as interactive as possible, however if you've got some suggestions for us then please let us know by the issues tab of this repo.

### Importing required modules/libraries

In [ ]:
import folium as flm
from folium import plugins
import pandas as pd
import json
import requests

### Loading Geo-JSON data
This data consists of co-ordinates which is used to plot the map.

In [ ]:
with open(r'/content/drive/MyDrive/Indian_States.json') as f:
    geojson_states = json.load(f)

### Assigning name of states to a key called 'id'

In [ ]:
for i in geojson_states['features']:
    i['id'] = i['properties']['NAME_1']

### Requesting data from API

In [ ]:
res = requests.get('https://api.covid19india.org/data.json')
covid_current = res.json()
df = []

### Filtering data for a better understanding
Such as renaming columns: 'state' to 'State', 'confirmed' to 'Total Case' and so on.

In [ ]:
for j in range(1, 37):
    if(covid_current['statewise'][j]['state'] != 'State Unassigned'):
        df.append([covid_current['statewise'][j]['state'],
                covid_current['statewise'][j]['confirmed'],
                covid_current['statewise'][j]['active'],
                covid_current['statewise'][j]['deaths'],
                ])
        df_covid = pd.DataFrame(df, columns=['State', 'Total Case', 'Active Case', 'Deaths'])

df_covid.head(5)

,State,Total Case,Active Case,Deaths
0,Maharashtra,1950171,49067,49759
1,Karnataka,923353,9637,12118
2,Andhra Pradesh,883587,3038,7122
3,Tamil Nadu,822370,7808,12177
4,Kerala,784489,63800,3185


### Filtering fetched data
Sorting data by states' name and renaming records to match with those in Geo-JSON file. For example, we changed data at index 7 to 'Dadra and Nagar Haveli' so that it can match with the data in Geo-JSON file. Initially, it was named as 'Daman and Diu Dadra and Nagar Haveli'.

In [ ]:
df_covid = df_covid.sort_values('State', axis=0)
df_covid = df_covid.reset_index(drop=True)

df_covid['State'].iloc[7] = 'Dadra and Nagar Haveli'

df_covid.head(5)

,State,Total Case,Active Case,Deaths
0,Andaman and Nicobar Islands,4948,38,62
1,Andhra Pradesh,883587,3038,7122
2,Arunachal Pradesh,16744,79,56
3,Assam,216381,1728,1057
4,Bihar,254689,4177,1415


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Exporting data to a CSV file

In [ ]:
#Q!mkfile t.csv
!pwd
df_covid.to_csv('/content/TotalCase.csv')

/content


### Reading CSV file for easier references

In [ ]:
pop_df = pd.read_csv('/content/TotalCase.csv')

### Adding fetched data from API to Geo-JSON file for later use

In [ ]:
for i in range(35):
    if((geojson_states['features'][i]['properties']['NAME_1']) == (df_covid['State'][i])):
        geojson_states['features'][i]['properties']['total_case'] = df_covid['Total Case'][i]
        geojson_states['features'][i]['properties']['active_case'] = df_covid['Active Case'][i]
        geojson_states['features'][i]['properties']['total_deaths'] = df_covid['Deaths'][i]     

### Creating a Folium Map Object

In [ ]:
map1 = flm.Map(location=[20.5937,78.9629], zoom_start=4, tiles='Mapbox Bright')

map1

### Adding more tiles to the map

In [ ]:
tiles = ['stamenwatercolor', 'cartodbpositron', 'openstreetmap', 'stamenterrain']
for tile in tiles:
    flm.TileLayer(tile).add_to(map1)

### Providing required data to the map for plotting
This is the most important step. Here we need to tell the map about the data - the Geo-JSON data and the data to be represented on the Geo-JSON data (i.e., COVID-19 data). We need to be very careful with the 'key_on' feature. (You can read more about them in the links provided at the end of this document).

In [ ]:
choropleth = flm.Choropleth(
    geo_data=geojson_states,
    name='Total Case',
    data=pop_df,
    columns=['State', 'Total Case'],
    key_on='feature.id',
    fill_color='Set2',
    fill_opacity=0.5,
    line_opacity=0.5,
    nan_fill_color='white',
    nan_fill_opacity=1, 
    line_weight=0.8,
    highlight=True,
    legend_name='State-wise COVID-19 Cases in INDIA',
	reset=True
    ).add_to(map1)

map1

### Adding some styling functions

In [ ]:
style_function = "font-size: 12px; font-weight: bold"
choropleth.geojson.add_child(
    flm.features.GeoJsonTooltip(fields=['NAME_1','total_case','active_case','total_deaths',], aliases=['State', 'Total Case', 'Active', 'Deaths'], lables=True, sticky=True, toLocaleString=True, style=style_function))

### Adding all the layers to map

In [ ]:
flm.LayerControl().add_to(map1)

### Finally rendering the map

In [ ]:
map1.save('IndianMap.html')
map1